In [4]:
%load_ext autoreload
%autoreload 2
import os
import glob
from PIL import Image
import numpy as np
current_dir = os.getcwd()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
def merge_image_lists(list1, list2, list3, output_path="merged_image.png"):
    """
    Merge three lists of images into one large image, where each row contains the images from one list.

    :param list1: List of image paths (row 1).
    :param list2: List of image paths (row 2).
    :param list3: List of image paths (row 3).
    :param output_path: Path to save the merged image.
    """
    # Combine all lists into one for processing
    image_lists = [list1, list2, list3]
    
    # Open all images in the lists
    image_arrays = [[Image.open(img_path) for img_path in image_list] for image_list in image_lists]
    print(image_arrays)
    
    # Ensure all images have the same dimensions
    widths, heights = zip(*(img.size for row in image_arrays for img in row))
    max_width = max(widths)
    max_height = max(heights)
    
    # Resize images to ensure uniformity
    image_arrays = [
        [img.resize((max_width, max_height)) for img in row] for row in image_arrays
    ]
    
    # Calculate the size of the final combined image
    total_width = max_width * len(list1)
    total_height = max_height * len(image_lists)
    
    # Create a blank canvas
    merged_image = Image.new("RGB", (total_width, total_height))
    
    # Paste each image onto the canvas
    for row_index, row_images in enumerate(image_arrays):
        for col_index, img in enumerate(row_images):
            x_offset = col_index * max_width
            y_offset = row_index * max_height
            merged_image.paste(img, (x_offset, y_offset))
    
    # Save the final image
    merged_image.save(output_path)
    print(f"Merged image saved to {output_path}")


def get_subdirs(path):
    try:
        subdirs = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
        return subdirs
    except FileNotFoundError:
        return []
    except PermissionError:
        return []

def get_flists(root_srcpath):
    tnames = ['Direct','T9','T8']
    out = []
    for tname in tnames:
        srcpath = os.path.join(root_srcpath,tname,'TMO')
        flist = glob.glob(os.path.join(srcpath,'*.tif'))
        out.append(flist)
    return out

root_srcpath = os.path.join(current_dir,'..',r'image\image_sondage')
snames = get_subdirs(root_srcpath)

for sname in snames[0:]:
    sub_root_srcpath = os.path.join(root_srcpath,sname)
    list1,list2,list3 = get_flists(sub_root_srcpath)
    dstfile = os.path.join(current_dir,sname+'.png')
    merge_image_lists(list1, list2, list3,dstfile)